# Broadcast Joins

Joining two (or more) data sources is an important and elementary operation in an relation algebra, like Spark. But actually the implementation is not trivial, especially for distributed systems like Spark. The main challenge is to physically bring together all records that need to be joined from both data sources onto a single machine, otherwise they cannot be merged. This means that data needs to be exchanged over the network, which is complex and slower than local access.

Depending on the size of the DataFrames to be joined, different strategies can be used. Spark supports two different join implementations:
* Shuffle join - will shuffle both DataFrames over the network to ensure that matching records end up on the same machine
* Broadcast join - will provide a copy of one DataFrames to all machines of the network

While shuffle joins can work with arbitrary large data sets, a broadcast join always requires that the broadcast DataFrame completely fits into memory on all machines. But it can be much faster when the DataFrame is small enoguh.

### Weather Example

Again we will investigate into the different join types with our weather example.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","24G") \
        .getOrCreate()

spark

/opt/anaconda3/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/25 16:58:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1 Load Data

First we load the weather data, which consists of the measurement data and some station metadata.

In [2]:
storageLocation = "s3://dimajix-training/data/weather"
# storageLocation = "/dimajix/data/weather-noaa-sample"

## 1.1 Load Measurements

Measurements are stored in multiple directories (one per year). But we will limit ourselves to a single year in the analysis to improve readability of execution plans.

In [3]:
from pyspark.sql.functions import *
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

Use a single year to keep execution plans small

In [4]:
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))

### Extract Measurements

Measurements were stored in a proprietary text based format, with some values at fixed positions. We need to extract these values with a simple SELECT statement.

In [5]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)

## 1.2 Load Station Metadata

We also need to load the weather station meta data containing information about the geo location, country etc of individual weather stations.

In [6]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# 2 Standard Joins

Per defaulkt Spark will automatically decide which join implementation to use (broadcast or hash exchange). In order to see the differences, we disable this automatic optimization and later we will explicitly instruct Spark how to perform a join.

With the automatic optimization disabled, all joins will be performed as hash exchange joins if not told otherwise.

In [7]:
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

## 2.1 Original Execution Plan

Let us have a look at the execution plan of the join.

In [8]:
df = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
df.explain(True)

== Parsed Logical Plan ==
Join Inner, ((usaf#100 = USAF#142) AND (wban#101 = WBAN#143))
:- Project [year#96, substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, substring(value#94, 16, 8) AS date#102, substring(value#94, 24, 4) AS time#103, substring(value#94, 42, 5) AS report_type#104, substring(value#94, 61, 3) AS wind_direction#105, substring(value#94, 64, 1) AS wind_direction_qual#106, substring(value#94, 65, 1) AS wind_observation#107, (cast(cast(substring(value#94, 66, 4) as float) as double) / cast(10.0 as double)) AS wind_speed#108, substring(value#94, 70, 1) AS wind_speed_qual#109, (cast(cast(substring(value#94, 88, 5) as float) as double) / cast(10.0 as double)) AS air_temperature#110, substring(value#94, 93, 1) AS air_temperature_qual#111]
:  +- Project [value#94, 2003 AS year#96]
:     +- Relation [value#94] text
+- Relation [USAF#142,WBAN#143,STATION NAME#144,CTRY#145,STATE#146,ICAO#147,LAT#148,LON#149,ELEV(M)#150,BEGIN#151,END#152] csv

== Analy

### Remarks

As said before, the join is a `SortMergeJoin` requiring a hash exchange shuffle operation. The join has the following steps:
1. Filter away `NULL` values (this is an inner join)
2. Repartition both DataFrames according to the join columns (`Exchange hashpartitioning`) with the same number of partitions each
3. Sort each partition of both DataFrames independently
4. Perform SortMergeJoin of both DataFrames by merging two according partitions from both DataFrames

This is a rather expensive operation, since it requires a repartitioning over network of both DataFrames.

## 2.2 Explicit Broadcast Joins

Now let us perform the logically same join operation, but this time using a *broadcast join* (also called *mapside join*). We can instruct Spark to broadcast a DataFrame to all worker nodes by using the `broadcast` function. This actually serves as a hint and returns a new DataFrame which is marked to be broadcasted in `JOIN` operations.

In [12]:
df = weather.join(broadcast(stations), ["usaf", "wban"])
df.explain()

== Physical Plan ==
*(2) Project [usaf#100, wban#101, 2003 AS year#96, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111, STATION NAME#144, CTRY#145, STATE#146, ICAO#147, LAT#148, LON#149, ELEV(M)#150, BEGIN#151, END#152]
+- *(2) BroadcastHashJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner, BuildRight, false
   :- *(2) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, substring(value#94, 16, 8) AS date#102, substring(value#94, 24, 4) AS time#103, substring(value#94, 42, 5) AS report_type#104, substring(value#94, 61, 3) AS wind_direction#105, substring(value#94, 64, 1) AS wind_direction_qual#106, substring(value#94, 65, 1) AS wind_observation#107, (cast(cast(substring(value#94, 66, 4) as float) as double) / 10.0) AS wind_speed#108, substring(value#94, 70, 1) AS wind_speed_qual#109, (cast(cast(substring(value

### Remarks

Now the execution plan looks significantly differnt. The stations metadata DataFrame is now broadcast to all worker nodes (still a network operation), but the measurement DataFrame does not require any repartitioning or shuffling any more. The broadcast join operation now is executed in three steps:
* Filter `NULL` values again
* Broadcast station metadata to all Spark executors
* Perform `BroadcastHashJoin`

A broadcast operation often makes sense in similar cases where you have large fact tables (measurements, purchase orders etc) and smaller lookup tables.

## 2.3 Full Outer Joins

Not all join types support broadcast joins. For example, full outer joins or right joins will not work

In [11]:
df = weather.join(broadcast(stations), ["usaf", "wban"], how="outer")
df.explain()

== Physical Plan ==
*(4) Project [coalesce(usaf#100, USAF#142) AS usaf#261, coalesce(wban#101, WBAN#143) AS wban#262, year#96, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111, STATION NAME#144, CTRY#145, STATE#146, ICAO#147, LAT#148, LON#149, ELEV(M)#150, BEGIN#151, END#152]
+- *(4) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], FullOuter
   :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=145]
   :     +- *(1) Project [2003 AS year#96, substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, substring(value#94, 16, 8) AS date#102, substring(value#94, 24, 4) AS time#103, substring(value#94, 42, 5) AS report_type#104, substring(value#94, 61, 3) AS wind_direction#105, substring(value#94, 64, 1) AS wind_

23/11/25 17:01:45 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.


## 2.4 Automatic Broadcast Joins

Per default Spark automatically determines which join strategy to use depending on the size of the DataFrames. This mechanism works fine when reading data from disk, but will not work after non-trivial transformations like `JOIN`s or grouped aggregations. In these cases Spark has no idea how large the results will be, but the execution plan has to be fixed before the first transformation is executed. In these cases (if by domain knowledge) you know that certain DataFrames will be small, an explicit `broadcast()` will still help.

### Reenable automatic broadcast

In order to re-enable Sparks default mechanism for selecting the `JOIN` strategy, we simply need to unset the configuration variable `spark.sql.autoBroadcastJoinThreshold`.

In [10]:
spark.conf.unset("spark.sql.autoBroadcastJoinThreshold")

### Inspect automatic execution plan

In [15]:
df = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
df.explain(True)

== Parsed Logical Plan ==
Join Inner, ((usaf#87 = USAF#122) && (wban#88 = WBAN#123))
:- Project [year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, substring(value#82, 16, 8) AS date#89, substring(value#82, 24, 4) AS time#90, substring(value#82, 42, 5) AS report_type#91, substring(value#82, 61, 3) AS wind_direction#92, substring(value#82, 64, 1) AS wind_direction_qual#93, substring(value#82, 65, 1) AS wind_observation#94, (cast(cast(substring(value#82, 66, 4) as float) as double) / cast(10.0 as double)) AS wind_speed#95, substring(value#82, 70, 1) AS wind_speed_qual#96, (cast(cast(substring(value#82, 88, 5) as float) as double) / cast(10.0 as double)) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
:  +- Project [value#82, 2003 AS year#84]
:     +- Relation[value#82] text
+- Relation[USAF#122,WBAN#123,STATION NAME#124,CTRY#125,STATE#126,ICAO#127,LAT#128,LON#129,ELEV(M)#130,BEGIN#131,END#132] csv

== Analyzed Logical Plan 

### Remarks

Since the stations metadata table is relatively small, Spark automatically decides to use a broadcast join again.